## 데이터 준비

파이토치에서는 데이터 준비를 위해 `torch.utils.data`의 `Dataset`과 `DataLoader` 사용 가능

- `Dataset`에는 다양한 데이터셋이 존재 (MNIST, FashionMNIST, CIFAR10, ...)
  - Vision Dataset: https://pytorch.org/vision/stable/datasets.html
  - Text Dataset: https://pytorch.org/text/stable/datasets.html
  - Audio Dataset: https://pytorch.org/audio/stable/datasets.html
- `DataLoader`와 `Dataset`을 통해 `batch_size`, `train` 여부, `transform` 등을 인자로 넣어 데이터를 어떻게 load할 것인지 정해줄 수 있음

In [1]:
import torch
from torch.utils.data import Dataset, DataLoader

토치비전(`torchvision`)은 파이토치에서 제공하는 데이터셋들이 모여있는 패키지

- `transforms`: 전처리할 때 사용하는 메소드 (https://pytorch.org/docs/stable/torchvision/transforms.html)
- `transforms`에서 제공하는 클래스 이외는 일반적으로 클래스를 따로 만들어 전처리 단계를 진행

In [2]:
import torchvision.transforms as transforms
from torchvision import datasets

`DataLoader`의 인자로 들어갈 `transform`을 미리 정의할 수 있고, `Compose`를 통해 리스트 안에 순서대로 전처리 진행

`ToTensor`()를 하는 이유는 `torchvision`이 PIL Image 형태로만 입력을 받기 때문에 데이터 처리를 위해서 Tensor형으로 변환 필요

In [3]:
mnist_transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize(mean=(0.5, ), std=(1.0, ))]) # 각 채널별로 평균 0.5, 표준편차 1.0으로 정규화

In [4]:
train_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=mnist_transform)
test_dataset = datasets.MNIST(root='./data', train=False, download=True, transform=mnist_transform)

100.0%


Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw



100.0%


Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw



100.0%


Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw



100.0%

Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw



`DataLoader`는 데이터 전체를 보관했다가 실제 모델 학습을 할 때 `batch_size` 크기만큼 데이터를 가져옴

In [5]:
train_loader = DataLoader(dataset=train_dataset, batch_size=8, shuffle=True, num_workers=2)
test_loader = DataLoader(dataset=test_dataset, batch_size=8, shuffle=False, num_workers=2)

In [6]:
data_iter = iter(train_loader)        # 반복자로 변환
images, labels = next(data_iter)      # 반복자로부터 데이터를 가져옴
images.shape, labels.shape            # 형태 확인(배치 크기, 채널 수, 높이, 너비), (배치 크기)  -> 8개의 이미지, 8개의 레이블

(torch.Size([8, 1, 28, 28]), torch.Size([8]))

In [7]:
torch_image = torch.squeeze(images[0]) # squeeze() 함수를 사용하여 채널 차원을 제거
torch_image.shape

torch.Size([28, 28])

In [8]:
import matplotlib.pyplot as plt 

figure = plt.figure(figsize=(12, 6))
cols, rows = 4, 2
for i in range(1, cols * rows + 1):
  sample_idx = torch.randint(len(train_dataset), size=(1, )).item()  # 0부터 len(train_dataset) - 1까지의 정수 중 하나를 랜덤하게 선택
  img, label = train_dataset[sample_idx]  # 랜덤하게 선택한 인덱스의 이미지와 레이블을 가져옴
  figure.add_subplot(rows, cols, i)       # 그림을 그릴 위치를 지정
  plt.title(label)                        # 레이블을 제목으로 설정
  plt.axis("off")                         # 축을 제거하여 이미지만 표시
  plt.imshow(img.squeeze(), cmap="gray")  # 이미지를 그림
plt.show()

ModuleNotFoundError: No module named 'kiwisolver'

## 신경망 구성

- `레이어(layer)`: 신경망의 핵심 데이터 구조로 하나 이상의 텐서를 입력받아 하나 이상의 텐서를 출력
- `모듈(module)`: 한 개 이상의 계층이 모여서 구성
- `모델(model)`: 한 개 이상의 모듈이 모여서 구성

In [9]:
import torch

### `torch.nn` 패키지

주로 가중치(weights), 편향(bias)값들이 내부에서 자동으로 생성되는 레이어들을 사용할 때 사용 (`weight`값들을 직접 선언 안함)

https://pytorch.org/docs/stable/nn.html

In [10]:
import torch.nn as nn

nn.Linear 계층 예제

In [11]:
input = torch.randn(128, 20)   # 128 -> 20  입력

m = nn.Linear(20, 30)          # 20 -> 30
print(m)

output = m(input)              # 128 -> 30  출력 
print(output)
print(output.size())

Linear(in_features=20, out_features=30, bias=True)
tensor([[ 1.0603,  0.8979, -0.1658,  ..., -0.8921, -0.3520,  0.0148],
        [ 0.1543, -0.2521, -0.0028,  ...,  0.4769,  0.0881,  0.3526],
        [-0.4344, -0.7889, -0.2943,  ..., -0.0602,  0.0767, -0.5576],
        ...,
        [-0.1146, -0.1986, -0.6528,  ...,  0.2428,  0.2309, -0.2106],
        [ 0.5082,  0.3665, -0.3046,  ..., -0.3219, -0.2938,  0.1732],
        [-0.1046,  0.8028, -0.4445,  ..., -0.2388, -0.0992, -0.3820]],
       grad_fn=<AddmmBackward0>)
torch.Size([128, 30])


`nn.Conv2d` 계층 예시

In [12]:
input = torch.randn(20, 16, 50, 100)
print(input.size())

torch.Size([20, 16, 50, 100])


In [13]:
m = nn.Conv2d(16, 33, 3, stride=2)      # 16 -> 33
m = nn.Conv2d(16, 33, (3, 5), stride=(2, 1), padding=(4, 2))  
m = nn.Conv2d(16, 33, (3, 5), stride=(2, 1), padding=(4, 2), dilation=(3, 1))
print(m)  

Conv2d(16, 33, kernel_size=(3, 5), stride=(2, 1), padding=(4, 2), dilation=(3, 1))


In [14]:
output = m(input)
print(output.size())

torch.Size([20, 33, 26, 100])


### 컨볼루션 레이어(Convolution Layers)

`nn.Conv2d` 예제

- `in_channels`: channel의 갯수
- `out_channels`: 출력 채널의 갯수
- `kernel_size`: 커널(필터) 사이즈

In [15]:
nn.Conv2d(1, 20, kernel_size=(5, 5), stride=(1, 1))

Conv2d(1, 20, kernel_size=(5, 5), stride=(1, 1))

In [16]:
layer = nn.Conv2d(1, 20, 5, 1).to(torch.device('cpu'))
layer

Conv2d(1, 20, kernel_size=(5, 5), stride=(1, 1))

weight 확인

In [17]:
weight = layer.weight
weight.size()

torch.Size([20, 1, 5, 5])

`weight`는 `detach()`를 통해 꺼내줘야 `numpy()`변환이 가능

In [18]:
weight = weight.detach().numpy()
weight.shape

(20, 1, 5, 5)

In [19]:
import matplotlib.pyplot as plt
plt.imshow(weight[0, 0, :, :], cmap='jet')
plt.colorbar()
plt.show()

ModuleNotFoundError: No module named 'kiwisolver'

In [20]:
# MNIST 데이터셋을 이용 
print(images.shape)
print(images[0].shape)
print(images[0].size())

input_image = torch.squeeze(images[0])
print(input_image.size())

torch.Size([8, 1, 28, 28])
torch.Size([1, 28, 28])
torch.Size([1, 28, 28])
torch.Size([28, 28])


In [21]:
input_data = torch.unsqueeze(images[0], dim=0)
print(input_data.size())

output_data = layer(input_data)
output = output_data.data
output_arr = output.numpy()
print(output_arr.shape)

torch.Size([1, 1, 28, 28])
(1, 20, 24, 24)


In [22]:
plt.figure(figsize=(15, 30))

plt.subplot(131)
plt.title('Input image')
plt.imshow(input_image, cmap='gray')

plt.subplot(132)
plt.title('Weight')
plt.imshow(weight[0, 0, :, :], cmap='jet')

plt.subplot(133)
plt.title('Output')
plt.imshow(output_arr[0, 0, :, :], cmap='gray')
plt.show()

NameError: name 'plt' is not defined

### 풀링 레이어(Pooling layers)

- `F.max_pool2d`
  - `stride`
  - `kernel_size`
- `torch.nn.MaxPool2d` 도 많이 사용